# Generate competitive versions 

Competitive version fulfill the following two aspects:
- Ca. 1 min 45 long (technically can be up to 2 minutes, but for now we do shorter finals)
- Fade out applied to the last 5 seconds

## Set up environment

In [57]:
import os
#from pathlib import Path
import pandas as pd

from pydub import AudioSegment

In [ ]:
# Import environment variables
from dotenv import load_dotenv

load_dotenv()
MUSIC_IN = os.getenv("MUSIC_IN")
SONGS_OUT = os.getenv("SONGS_OUT")
FINALS_OUT = os.getenv("FINALS_OUT")

In [59]:
# Write some utility functions
def get_song_path(song_name):
    '''
    In my current setup, I only need to copy the file name but not the directory. The directory is always a subfolder of MUSIC_IN. This function searches for the song in the subfolders and returns the full path to the songs.
    '''
    for root, dirs, files in os.walk(MUSIC_IN):
        if song_name in [file_name.split(".")[0] for file_name in files]:
            original_song_name = [file_name for file_name in files if file_name.split(".")[0] == song_name][0]
            return os.path.join(root, original_song_name)

## Import Songs

Songs are saved in the way:

- "." separates Latin and Standard (i.e., ". Latin" begins the Latin category)
- "+" separates dances (i.e., "+ Slow Waltz" begins the Slow Waltz list)
- New lines separate the different songs

The code below uses string splitting to generate the corresponding list and save the information in an organized fashion to a dataframe

In [60]:
# Read file
with open("list_songs.txt", "r") as songs_file:
    content = songs_file.read()
# Get number of lines
number_lines = content.count("\n") + 1
number_songs = number_lines - 10 - 2

list_rows = []

list_types = content.split(".")[1:]
# Generate list of lists for later use in pandas
# Iterate over dance types (Standard vs Latin)
for list_type in list_types:
    list_dances = list_type.split("+")
    dance_type_name = list_type.split("\n")[0]
    # Iterate over dances
    for dance_type in list_dances:
        list_songs = dance_type.split("\n")
        # Loop over songs
        for i in range(1, len(list_songs)-1):
            song_file_name = list_songs[i].lstrip()
            song_file_path = get_song_path(song_file_name)
            if song_file_path is None:
                print(f"Song {song_file_name} not found in {MUSIC_IN}")
            list_rows.append([dance_type_name.strip(), list_songs[0].strip(), song_file_name, song_file_path])

# Create dataframe
dataframe_songs = pd.DataFrame(list_rows, columns = ["type", "dance", "song_name", "song_path"])

## Import songs and make them competition version

In [61]:
for index, row in dataframe_songs.iterrows():
    song = AudioSegment.from_file(row["song_path"])
    if row["dance"] == "Paso Doble":
        # Espana Cani is as always the exception
        song_part = song[:(1000 * (60 + 25))]
    else:    
        song_part = song[:(1000 * 1.75 * 60)]
        song_part = song_part.fade_out(5 * 1000)
    song_part.export(os.path.join(SONGS_OUT, "competition - " + row["song_name"] + ".mp4"), format="mp4")

# Create finals

In [62]:
# Order of songs
song_list_standard = ["Slow Waltz", "Tango", "Viennese Waltz", "Slowfox" , "Quickstep"]
song_list_latin = ["Samba", "Cha", "Rumba", "Paso Doble", "Jive"]

break_segment = AudioSegment.silent(duration=1000 * 20)

number_finals = 4
for type_dance in ["Standard", "Latin"]:
    song_counter = 0
    if type_dance == "Standard":
        song_list = song_list_standard
    elif type_dance == "Latin":
        song_list = song_list_latin
    else:
        raise ValueError("Unknown dance type")
    for finals_i in range(0, number_finals):
        final_song = AudioSegment.empty()
        candidate_songs = dataframe_songs[dataframe_songs["type"] == type_dance]["song_name"].tolist()
        for dance in song_list:
            candidate_songs_type =  dataframe_songs[dataframe_songs["dance"] == dance]["song_name"].tolist()
            song_number = (song_counter % (len(candidate_songs_type))) # Ensures that we loop back if we want more finals than we have songs
            current_song = candidate_songs_type[song_number]
            final_song += AudioSegment.from_file(os.path.join(SONGS_OUT, "competition - " + current_song + ".mp4"))
            final_song += break_segment
        song_counter += 1
        final_song.export(os.path.join(FINALS_OUT, "finals - " + type_dance + " - " + str(finals_i+1) + ".mp4"), format="mp4")
        